# SB79 Transit Oriented Development Upzoning

## Project Resources

- [Pedestrian Access Points](https://mtcdrive.box.com/s/q33u23k3amzgyidcf25cp1lzz6fn8br9)
- [Transit Stations](https://mtcdrive.box.com/s/jafqtaxw419tmw0r0m5z7j6g53l9b1p1)
- [511 GTFS Data](https://mtcdrive.box.com/s/qro5h0uvwwyovx1iuhvcjy55bjqbuana)
- [San Francisco Bay Region High Quality Transit Stop Data](https://mtc.maps.arcgis.com/home/item.html?id=981ce33db7714f74b126489ef733437b)
- [San Francisco Bay Region Jurisdiction Boundaries]()

## Methodology

Load Data and Libraries
1. Set up environment and load necessary libraries (e.g., pandas, geopandas, gtfs_kit)
2. Load GTFS data using gtfs_kit and convert to GeoDataFrame for spatial analysis
3. Load pedestrian access point data and transit station data as GeoDataFrames
4. Load Caltrans High Quality Transit Stops (HQTS) data as GeoDataFrame

Prepare Data for Analysis
1. Enrich GTFS stops with route and agency information by merging with routes and agency tables; aggregate to get lists of routes and agencies serving each stop
2. Filter GTFS stops to include only those served by relevant transit agencies (e.g., BART, Caltrain, AC Transit, SFMTA, VTA)
3. Separate stations and stops based on GTFS hierarchy based on location_type and parent_station fields (see [GTFS documentation](https://gtfs.org/documentation/schedule/reference/#stopstxt) for details)
4. Filter Caltrans High Quality Station Stops to include only those with `HQTA Type == major_stop_brt` 

Flag Transit-Oriented Development (TOD) Stops
1. Flag GTFS stops as TOD applicable if they are either:
   1. A stop that meets the Caltrans definition of a major BRT stop (i.e., in the filtered Caltrans dataset) using a list of `stop_id`s from the filtered HQTS dataset, OR
   2. A stop with `route_type in [0 (Tram, Streetcar, Light rail), 1 (Subway, Metro)]` (see [GTFS documentation](https://gtfs.org/documentation/schedule/reference/#routestxt) for details) or `agency_id == 'CT' (Caltrain)` 
      1. Exceptions: Exclude stops South of Tamien Station on the Caltrain System, and BART stops within Contra Costa County. 
2. Create Transit Tier classification:
   1. Tier 1: `route_type == 1 (Subway, Metro)` or `agency_id == 'CT' (Caltrain)`
   2. Tier 2: `route_type == 0 (Tram, Streetcar, Light rail)` or a stop that meets the Caltrans definition of a major BRT stop (i.e., in the filtered Caltrans dataset) using a list of `stop_id`s from the filtered HQTS dataset.
3. Create a final filtered GeoDataFrame of TOD applicable stops, including relevant attributes such as stop_id, stop_name, parent_station, agency_name, route_short_name, and geometry.

Associate Parent Stations with TOD Stops & Access Points
1.  Manually create stations for stops that are flagged as TOD applicable, such as SFMTA light rail stops not co-located with a BART Station (e.g. Van Ness, Church, Forest Hill, Yerba Buena/Moscone, etc.), VTA light rail stops, and BRT stops. 
2.  Manually create pedestrian access points for stops that are flagged as TOD applicable following the same process as above (add guidance from HCD on what constitutes a pedestrian access point, e.g. crosswalks, sidewalks, etc.).
3.  Associate stops and access points with parent stations. This may be performed by spatially joining stops and access points to stations using a near spatial join with a specified distance threshold (e.g. 200 feet) though manual review and adjustments will likely be necessary to ensure accurate associations, especially in dense urban areas where multiple stations and stops may be in close proximity.

Create Transit-Oriented Development (TOD) Zones 

**SB79 TOD Zones – Applicability Matrix**

This table summarizes where Transit-Oriented Development (TOD) Zones apply by Transit Tier and Jurisdiction Type. Distances are measured from pedestrian access points and represent distinct distance bands.

Where TOD Zones Apply

| Tier       | City with Population > 35,000       | City with Population ≤ 35,000 | Unincorporated Area |
| ---------- | ----------------------------------- | ----------------------------- | ------------------- |
| **Tier 1** | 0–200’<br>201’–1320’<br>1321’–2640’ | 0–200’<br>201’–1320’          | Not Applicable      |
| **Tier 2** | 0–200’<br>201’–1320’<br>1321’–2640’ | 0–200’<br>201’–1320’          | Not Applicable      |

Notes

1. Tier Precedence Rule:
   - Where Tier 1 and Tier 2 zones intersect, Tier 1 supersedes Tier 2. Tier 2 geometry must be erased in overlapping areas.
2. Geographic Scope:
   - Applies only to cities located within:
     - Alameda County
     - San Francisco County
     - San Mateo County
     - Santa Clara County
3. Distance Bands:
   - Bands are distinct (non-cumulative):
     - 0–200 feet
     - 201–1320 feet (¼ mile ring excluding first 200 feet)
     - 1321–2640 feet (½ mile ring excluding first ¼ mile)

Implementation Steps
1. Generate 200 ft, .25 mile, and .5 mile Euclidean (straight-line) buffers around all pedestrian access points by tier
2. Intersect buffers with jurisdiction boundaries with associated population data to determine applicable zones based on the matrix above
3. Remove .5 mile buffers where jurisdiction population is ≤ 35,000 or in unincorporated areas
4. Erase Tier 2 areas where overlapping with Tier 1 buffers to ensure Tier 1 precedence
5. Dissolve by Tier and distance band to create final TOD Zone geometries
6. Validate and review final TOD Zone geometries for accuracy and consistency with the defined criteria 




In [1]:
import getpass
import gtfs_kit as gk
import pandas as pd
import geopandas as gpd
from pathlib import Path

# import user 
user = getpass.getuser()

In [2]:
# set up file paths
data_dir = Path(f"/Users/{user}/Library/CloudStorage/Box-Box/DSA Projects/Spatial Analysis and Mapping/SB79 Transit Oriented Development/Data")
gtfs_dir = data_dir / "GTFSTransitData_RG_2026_02_18.zip"
access_pts_dir = data_dir / "Access Points"
stations_dir = data_dir / "Stations"

In [3]:
# read in data 
feed = gk.read_feed(gtfs_dir, dist_units='mi')
stops_gdf = gk.geometrize_stops(feed.stops)
ac_gdf = gpd.read_file(access_pts_dir / "ACTransit_PedAccessPoints_GTFS.zip")
bart_gdf = gpd.read_file(access_pts_dir / "BART_PedAccessPoints_GTFS_v1.zip")
caltrain_gdf = gpd.read_file(access_pts_dir / "Caltrain_PedAccessPoints_GTFS_v3.zip")
vta_gdf = gpd.read_file(access_pts_dir / "VTA_PedAccessPoints_GTFS_v2.zip")

In [4]:
# Enrich stops with agency and route information
# Join through GTFS relational structure: stops → stop_times → trips → routes → agency

# Get unique stop-route relationships
stop_routes = (
    feed.stop_times[["stop_id", "trip_id"]]
    .merge(feed.trips[["trip_id", "route_id"]], on="trip_id")
    .drop_duplicates(["stop_id", "route_id"])
)

# Add route details
stop_routes = stop_routes.merge(
    feed.routes[["route_id", "agency_id", "route_short_name", "route_long_name", "route_type"]],
    on="route_id",
)

# Add agency details
stop_routes = stop_routes.merge(
    feed.agency[["agency_id", "agency_name"]], on="agency_id", how="left"
)

# Filter to only relevant transit agencies for this project
relevant_agencies = ['BA', 'CT', 'AC', 'SC', 'SF']
stop_routes = stop_routes[stop_routes['agency_id'].isin(relevant_agencies)]

# Aggregate by stop_id to get lists of agencies and routes per stop
stop_info = (
    stop_routes.groupby("stop_id")
    .agg(
        {
            "agency_name": lambda x: ", ".join(sorted(set(x.dropna()))),
            "agency_id": lambda x: ", ".join(sorted(set(x.dropna()))),
            "route_short_name": lambda x: ", ".join(sorted(set(x.dropna()))),
            "route_long_name": lambda x: ", ".join(sorted(set(x.dropna()))),
            "route_type": lambda x: ", ".join(map(str, sorted(set(x.dropna())))),
        }
    )
    .reset_index()
)

# Merge with stops_gdf
stops_gdf = stops_gdf.merge(stop_info, on="stop_id", how="left")

In [5]:
feed.agency[["agency_id", "agency_name"]]

,agency_id,agency_name
0,UC,Union City Transit
1,PE,PETALUMA
2,PG,Presidio Go
3,EE,Emery Express
4,MC,Mountain View Community Shuttle
5,SR,SANTAROSA
6,SI,San Francisco International Airport
7,FS,FAST
8,AF,Angel Island Tiburon Ferry
9,CM,Commute.org Shuttles


In [6]:
# Separate stations and stops based on GTFS hierarchy

# Create stations dataset (location_type = 1)
# These are parent structures that contain stops
stations_gdf = stops_gdf[stops_gdf['location_type'] == 1][['stop_id', 'stop_name', 'location_type', 'geometry']].copy()

# Create stops dataset (location_type = 0 or blank)
# Filter to only stops that are actually served by routes (have route info)
stops_gdf = stops_gdf[
    (stops_gdf['location_type'].fillna(0) == 0) & 
    (stops_gdf['route_short_name'].notna())
].copy()

# Clean up stops_gdf: remove unnecessary columns
cols_to_drop = ['stop_code', 'stop_desc', 'zone_id', 'stop_url', 'tts_stop_name', 
                'platformcode', 'stop_timezone', 'wheelchair_boarding', 'route_long_name']
stops_gdf = stops_gdf.drop(columns=[col for col in cols_to_drop if col in stops_gdf.columns])

# Move geometry to the end
cols = [col for col in stops_gdf.columns if col != 'geometry'] + ['geometry']
stops_gdf = stops_gdf[cols]

print(f"Total records in original data: {len(stops_gdf) + len(stations_gdf)}")
print(f"Stations (location_type=1): {len(stations_gdf)}")
print(f"Stops (location_type=0, served by routes): {len(stops_gdf)}")
print(f"Stops with parent_station: {stops_gdf['parent_station'].notna().sum()}")
print(f"\nColumns in stops dataset: {list(stops_gdf.columns)}")

Total records in original data: 11658
Stations (location_type=1): 301
Stops (location_type=0, served by routes): 11357
Stops with parent_station: 423

Columns in stops dataset: ['stop_id', 'stop_name', 'platform_code', 'location_type', 'parent_station', 'level_id', 'agency_name', 'agency_id', 'route_short_name', 'route_type', 'geometry']


In [7]:
# View sample of stations
print("Sample stations:")
stations_gdf.head(10)

Sample stations:


,stop_id,stop_name,location_type,geometry
0,mtc:powell,Powell,1,POINT (-122.40737 37.78459)
1,mtc:fruitvale,Fruitvale,1,POINT (-122.22412 37.77486)
2,mtc:warm-springs-south-fremont-bart,Warm Springs South Fremont BART,1,POINT (-121.93925 37.50199)
3,mtc:caltrain-4th-&-king,Caltrain 4th & King,1,POINT (-122.39487 37.77649)
4,mtc:el-cerrito-del-norte-bart,El Cerrito Del Norte BART,1,POINT (-122.31699 37.92529)
5,mtc:walnut-creek-bart,Walnut Creek BART,1,POINT (-122.06734 37.9058)
6,mtc:richmond-bart-amtrak,Richmond BART/Amtrak,1,POINT (-122.35314 37.93684)
7,mtc:santa-clara-caltrain,Santa Clara Caltrain,1,POINT (-121.93555 37.35313)
8,mtc:san-francisco-ferry-terminal,San Francisco Ferry Terminal,1,POINT (-122.39341 37.79553)
9,mtc:macarthur-bart,MacArthur BART,1,POINT (-122.26706 37.82909)


In [8]:
stops_gdf

,stop_id,stop_name,platform_code,location_type,parent_station,level_id,agency_name,agency_id,route_short_name,route_type,geometry
306,16995,Metro Powell Station/Outbound,<NA>,0,mtc:powell,mtc:powell-muni-platform,San Francisco Municipal Transportation Agency,SF,"J, K, L, M, N",0,POINT (-122.40782 37.7843)
307,15417,Metro Powell Station/Downtown,<NA>,0,mtc:powell,mtc:powell-muni-platform,San Francisco Municipal Transportation Agency,SF,"J, K, L, M, N",0,POINT (-122.40709 37.78465)
308,51333,Fruitvale BART,<NA>,0,mtc:fruitvale,mtc:fruitvale-street-level,AC TRANSIT,AC,"19, 51A, 851",3,POINT (-122.22457 37.775)
309,59000,Fruitvale BART,<NA>,0,mtc:fruitvale,mtc:fruitvale-street-level,AC TRANSIT,AC,"30, 31, 639",3,POINT (-122.22474 37.77511)
310,55550,Fruitvale BART,<NA>,0,mtc:fruitvale,mtc:fruitvale-street-level,AC TRANSIT,AC,"14, 54",3,POINT (-122.22489 37.7752)
...,...,...,...,...,...,...,...,...,...,...,...
15432,70042,South San Francisco Caltrain Station Southbound,<NA>,0,south_sf,<NA>,Caltrain,CT,"Express, Limited, Local Weekday, Local Weekend",2,POINT (-122.40498 37.65594)
15435,70221,Sunnyvale Caltrain Station Northbound,<NA>,0,sunnyvale,<NA>,Caltrain,CT,"Express, Limited, Local Weekday, Local Weekend",2,POINT (-122.03137 37.37892)
15436,70222,Sunnyvale Caltrain Station Southbound,<NA>,0,sunnyvale,<NA>,Caltrain,CT,"Express, Limited, Local Weekday, Local Weekend",2,POINT (-122.03142 37.37879)
15437,70271,Tamien Caltrain Station Northbound,<NA>,0,tamien,<NA>,Caltrain,CT,"Local Weekday, Local Weekend, South County",2,POINT (-121.88372 37.31174)


In [ ]:
stops_gdf.agency_name.value_counts()

agency_name
AC TRANSIT                                       4704
VTA                                              3246
San Francisco Municipal Transportation Agency    3244
Bay Area Rapid Transit                            103
Caltrain                                           60
Name: count, dtype: int64[pyarrow]

In [ ]:
# View sample of served stops with route/agency information
print("Sample stops with route/agency info:")
stops_gdf[['stop_id', 'stop_name', 'parent_station', 'agency_name', 'route_short_name', 'geometry']].head(10)

Sample stops with route/agency info:


,stop_id,stop_name,parent_station,agency_name,route_short_name,geometry
306,16995,Metro Powell Station/Outbound,mtc:powell,San Francisco Municipal Transportation Agency,"J, K, L, M, N",POINT (-122.40782 37.7843)
307,15417,Metro Powell Station/Downtown,mtc:powell,San Francisco Municipal Transportation Agency,"J, K, L, M, N",POINT (-122.40709 37.78465)
308,51333,Fruitvale BART,mtc:fruitvale,AC TRANSIT,"19, 51A, 851",POINT (-122.22457 37.775)
309,59000,Fruitvale BART,mtc:fruitvale,AC TRANSIT,"30, 31, 639",POINT (-122.22474 37.77511)
310,55550,Fruitvale BART,mtc:fruitvale,AC TRANSIT,"14, 54",POINT (-122.22489 37.7752)
311,55355,Fruitvale BART,mtc:fruitvale,AC TRANSIT,O,POINT (-122.22521 37.77523)
312,53111,Fruitvale BART,mtc:fruitvale,AC TRANSIT,"19, 648, 654, 655",POINT (-122.22537 37.77533)
313,55533,Fruitvale BART,mtc:fruitvale,AC TRANSIT,"30, 31, 639",POINT (-122.22511 37.77534)
314,55522,Fruitvale BART,mtc:fruitvale,AC TRANSIT,"62, 703",POINT (-122.22528 37.77543)
315,55886,Fruitvale BART,mtc:fruitvale,AC TRANSIT,"14, 54, 62",POINT (-122.22546 37.77552)


In [ ]:
stops_gdf.head()

,stop_id,stop_name,platform_code,location_type,parent_station,level_id,agency_name,agency_id,route_short_name,route_type,geometry
306,16995,Metro Powell Station/Outbound,<NA>,0,mtc:powell,mtc:powell-muni-platform,San Francisco Municipal Transportation Agency,SF,"J, K, L, M, N",0,POINT (-122.40782 37.7843)
307,15417,Metro Powell Station/Downtown,<NA>,0,mtc:powell,mtc:powell-muni-platform,San Francisco Municipal Transportation Agency,SF,"J, K, L, M, N",0,POINT (-122.40709 37.78465)
308,51333,Fruitvale BART,<NA>,0,mtc:fruitvale,mtc:fruitvale-street-level,AC TRANSIT,AC,"19, 51A, 851",3,POINT (-122.22457 37.775)
309,59000,Fruitvale BART,<NA>,0,mtc:fruitvale,mtc:fruitvale-street-level,AC TRANSIT,AC,"30, 31, 639",3,POINT (-122.22474 37.77511)
310,55550,Fruitvale BART,<NA>,0,mtc:fruitvale,mtc:fruitvale-street-level,AC TRANSIT,AC,"14, 54",3,POINT (-122.22489 37.7752)
